In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import scipy.signal
import hdf5storage

# GPS file path
gps_mat_file = "outputs/tmp/gps_test.mat"
gps_tmp_df_file = "outputs/tmp/gps_test.csv"

# Load the GPS .mat file
gps_data = hdf5storage.loadmat(gps_mat_file)

# Load the GPS temporary CSV file
gps_tmp_df = pd.read_csv(gps_tmp_df_file)

In [ ]:
print("==== gps_data (from .mat) keys ====")
print(gps_data.keys())

print("\n==== gps_tmp_df (from .csv) columns ====")
print(gps_tmp_df.columns)

In [ ]:
expected_matching_keys = [('gps_time', 'GPS_TIME'), ('lat', 'LAT'), ('lon', 'LON'), ('elev', 'ELEV'), ('heading', 'HEADING'), ('roll', 'ROLL'), ('pitch', 'PITCH')]

for gps_key, df_key in expected_matching_keys:
    assert gps_key in gps_data, f"Key '{gps_key}' not found in gps_data"
    assert df_key in gps_tmp_df.columns, f"Column '{df_key}' not found in gps_tmp_df"
    assert len(gps_data[gps_key].squeeze()) == len(gps_tmp_df[df_key]), f"Length mismatch for '{gps_key}' and '{df_key}'"
    assert np.allclose(gps_data[gps_key].squeeze(), gps_tmp_df[df_key], atol=1e-4), f"Data mismatch for '{gps_key}' and '{df_key}'"

In [ ]:
wrap_keys = ['HEADING_field_imu']
for k in wrap_keys:
    gps_tmp_df[k+"_wrapped"] = (gps_tmp_df[k] + np.pi) % (2 * np.pi) - np.pi

In [ ]:
(gps_tmp_df['gps_time_field_gps'] - gps_tmp_df['pps_time_field_gps']).mean()

In [ ]:
plot_keys = [
    (('gps_time', ['lat']),
     ('GPS_TIME', ['LAT', 'LAT_field_gps'])),
    (('gps_time', ['lon']),
     ('GPS_TIME', ['LON', 'LON_field_gps'])),
    (('gps_time', ['roll']),
     ('GPS_TIME', ['ROLL', 'ROLL_field_imu'])),
    (('gps_time', ['pitch']),
     ('GPS_TIME', ['PITCH', 'PITCH_field_imu'])),
    (('gps_time', ['heading']),
     ('GPS_TIME', ['HEADING', 'HEADING_field_imu_wrapped'])),
    (('gps_time', ['elev']),
     ('GPS_TIME', ['ELEV', 'vert_cor_field_gps'])),
    (('gps_time', []),
     ('GPS_TIME', ['gps_time_field_gps', 'pps_time_field_gps']))
]

fig, ax = plt.subplots(len(plot_keys), 1, figsize=(10, 5 * len(plot_keys)))

n_max = 10000

if len(plot_keys) == 1:
    ax = [ax]  # Ensure ax is always a list for consistent indexing

for i, ((mat_key, mat_subkeys), (csv_key, csv_subkeys)) in enumerate(plot_keys):
    if len(mat_subkeys) > 0:
        ax[i].plot(gps_data[mat_key].flatten()[:n_max], gps_data[mat_subkeys[0]].flatten()[:n_max], label=f'MATLAB {mat_subkeys[0]}', color='blue')
    for subkey in csv_subkeys:
        ax[i].plot(gps_tmp_df[csv_key][:n_max], gps_tmp_df[subkey][:n_max], label=f'CSV {subkey}', linestyle='--')
    ax[i].set_xlabel(f"{mat_key} / {csv_key}")
    ax[i].legend()
    ax[i].grid()


In [ ]:
def plot_xcorr(x1, y1, x2, y2, ax, label=None, max_time=None):
    # Determine the smaller x spacing
    dx1 = np.mean(np.diff(x1))
    dx2 = np.mean(np.diff(x2))
    dx = min(dx1, dx2)
    # Create a common x axis
    x_start = max(min(x1), min(x2))
    x_stop = min(max(x1), max(x2))
    if max_time:
        x_stop = min(x_stop, x_start + max_time)
    x_common = np.arange(x_start, x_stop, dx)
    # Interpolate y1 and y2 onto the common x axis
    y1_interp = np.interp(x_common, x1, y1)
    y2_interp = np.interp(x_common, x2, y2)
    # Compute the cross-correlation
    corr = scipy.signal.correlate(y1_interp - np.mean(y1_interp), y2_interp - np.mean(y2_interp), mode='full')
    lags = scipy.signal.correlation_lags(len(y1_interp), len(y2_interp), mode='full')
    lags = lags * dx
    # Normalize the cross-correlation
    corr = corr / np.max(np.abs(corr))

    # Plot the cross-correlation
    ax.plot(lags, corr, label=label)
    ax.set_xlabel('Lag [s]')
    ax.set_ylabel('Cross-correlation')
    
    # Put a dot and label the peak (max or min)
    peak_idx = np.argmax(np.abs(corr))
    ax.plot(lags[peak_idx], corr[peak_idx], 'ro')
    ax.text(lags[peak_idx], corr[peak_idx], f'Peak at {lags[peak_idx]:.2f}s', fontsize=10, verticalalignment='bottom')
    ax.grid()

def copy_axes(source_ax, target_ax):
    """Copy all artists from source axes to target axes"""
    
    # Copy lines
    for line in source_ax.get_lines():
        target_ax.add_line(plt.Line2D(line.get_xdata(), line.get_ydata(),
                                       color=line.get_color(),
                                       linewidth=line.get_linewidth(),
                                       linestyle=line.get_linestyle(),
                                       marker=line.get_marker(),
                                       label=line.get_label()))
    
    # Copy patches (rectangles, circles, etc.)
    for patch in source_ax.patches:
        import copy
        target_ax.add_patch(copy.copy(patch))
    
    # Copy collections (scatter, etc.)
    for collection in source_ax.collections:
        target_ax.add_collection(copy.copy(collection))
    
    # Copy texts
    for text in source_ax.texts:
        target_ax.text(text.get_position()[0], text.get_position()[1],
                      text.get_text())
    
    # Copy axis labels and title
    target_ax.set_xlabel(source_ax.get_xlabel())
    target_ax.set_ylabel(source_ax.get_ylabel())
    target_ax.set_title(source_ax.get_title())
    
    # Copy axis limits
    target_ax.set_xlim(source_ax.get_xlim())
    target_ax.set_ylim(source_ax.get_ylim())
    
    # Copy legend if it exists
    if source_ax.get_legend():
        target_ax.legend()

In [ ]:
max_time = 60*30

fig, (ax, ax_zoom) = plt.subplots(2, 1, figsize=(10, 10))

# Latitude

plot_xcorr(
    gps_data['gps_time'].squeeze(), gps_data['lat'].squeeze(),
    gps_tmp_df['GPS_TIME'], gps_tmp_df['LAT_field_gps'],
    ax, label="gps_data['lat'] vs gps_tmp_df['LAT_field_gps']",
    max_time=max_time
)

# Roll

plot_xcorr(
    gps_data['gps_time'].squeeze(), gps_data['roll'].squeeze(),
    gps_tmp_df['GPS_TIME'], gps_tmp_df['ROLL_field_imu'],
    ax, label="gps_data['roll'] vs gps_tmp_df['ROLL_field_imu']",
    max_time=max_time
)

# Heading

plot_xcorr(
    gps_data['gps_time'].squeeze(), gps_data['heading'].squeeze(),
    gps_tmp_df['GPS_TIME'], gps_tmp_df['HEADING_field_imu_wrapped'],
    ax, label="gps_data['heading'] vs gps_tmp_df['HEADING_field_imu_wrapped']",
    max_time=max_time
)

# Elevation

plot_xcorr(
    gps_data['gps_time'].squeeze(), gps_data['elev'].squeeze(),
    gps_tmp_df['GPS_TIME'], gps_tmp_df['vert_cor_field_gps'],
    ax, label="gps_data['elev'] vs gps_tmp_df['vert_cor_field_gps']",
    max_time=max_time
)

# gps_time_field_gps vs pps_time_field_gps
plot_xcorr(
    gps_tmp_df['GPS_TIME'], gps_tmp_df['gps_time_field_gps'],
    gps_tmp_df['GPS_TIME'], gps_tmp_df['pps_time_field_gps'],
    ax, label="gps_tmp_df['gps_time_field_gps'] vs gps_tmp_df['pps_time_field_gps']",
    max_time=max_time
)

ax.grid(True)
ax.legend()
#ax.set_xlim(-120, 120)
ax.set_title(f'Cross-correlations within CSV file [max time window = {max_time} s]')

# Copy the last plot to the zoomed-in axes
copy_axes(ax, ax_zoom)
ax_zoom.grid(True)
ax_zoom.set_xlim(-30, 30)
ax_zoom.set_ylim(0.5, 1.1)

In [ ]:
from utig_radar_loading.opr_gps_file_generation import load_and_parse_postprocessed_gps_file

df = load_and_parse_postprocessed_gps_file("/resfs/GROUPS/CRESIS/dataproducts/metadata/2018_Antarctica_BaslerJKB/gps/EPUTG1B_J014_PEL_JKB2u_Y20b_position.txt")
df['GPS_TIME'].diff().iloc[:100].plot()